**Problem Tanımı:**


ABC Şehri'ndeki bir kavşakta trafik ışıklarının kontrol sürelerini optimize etmek ve kavşaktaki araç bekleme sürelerini minimize etmek istiyoruz. Bu kavşak, sabah ve akşam saatlerinde yoğun trafik akışının olduğu bir bölgede yer almaktadır. Mevcut durumda, trafik ışıkları sabit sürelerle çalışmakta ve araçlar sık sık uzun süreler boyunca beklemektedir. Bu durum, trafiği yavaşlatmakta, stresli sürüş deneyimine neden olmakta ve yakıt tüketimini artırmaktadır.

Bu sorunu çözmek için, bulanık mantık yöntemlerini kullanarak trafik ışıklarının kontrol sürelerini dinamik bir şekilde ayarlamayı hedefliyoruz. Bu şekilde, kavşaktaki araç yoğunluğuna ve trafik akışına bağlı olarak trafik ışıklarının yeşil süresini optimize edebiliriz. Böylece, araç bekleme sürelerini minimize ederek trafik akışını düzenleyebilir ve araçların daha hızlı ve güvenli bir şekilde kavşaktan geçmesini sağlayabiliriz.



**Girdi Değişkenleri**

1.Araç Yoğunluğu (car_density): Kavşaktaki araç sayısının bir ölçüsüdür. Bu değişken, 0 ile 10 arasında sürekli bir değer alır. Daha yüksek değerler, kavşaktaki araç yoğunluğunun arttığını gösterir.


2.Trafik Akışı (traffic_flow): Kavşaktaki trafik akışının bir ölçüsüdür. Bu değişken, 0 ile 10 arasında sürekli bir değer alır. Daha yüksek değerler, kavşaktaki trafik akışının arttığını gösterir.

**Çıktı Değişkeni**

Trafik Işıklarının Kontrol Süresi (signal_duration): Bu değişken, trafik ışıklarının yeşil ışık süresini belirlemek için kullanılır. 0 ile 30 arasında sürekli bir değer alır. Daha yüksek değerler, yeşil ışık süresinin uzadığını gösterir.

**Kısıtlar**

1.Araç yoğunluğu ve trafik akışı, 0 ile 10 arasında bir değer almalıdır.


2.Trafik ışıklarının kontrol süresi, 0 ile 30 arasında bir değer almalıdır.

**Amaç fonksiyonu**

1.Araç bekleme sürelerini minimize etmek: Trafik akışını optimize etmek ve kavşakta araç bekleme sürelerini mümkün olduğunca azaltmak hedeflenir.


2.Trafik akışını optimize etmek: Trafik akışını düzenlemek ve araç trafiğinin daha akıcı olmasını sağlamak amaçlanır.


3.Yaya trafiği için uygun geçiş sürelerini sağlamak: Yaya trafiğine uygun geçiş süreleri tanımlamak ve yaya güvenliğini ön planda tutmak hedeflenir.

**W_max = α * (1 - V) + β * (1 - F)**




In [7]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Girdi değişkenlerini tanımlama
car_density = ctrl.Antecedent(np.arange(0, 11, 1), 'car_density')
traffic_flow = ctrl.Antecedent(np.arange(0, 11, 1), 'traffic_flow')

# Çıktı değişkenini tanımlama
signal_duration = ctrl.Consequent(np.arange(0, 31, 1), 'signal_duration')

# Girdi değişkenleri için üyelik fonksiyonlarını tanımlama
car_density['low'] = fuzz.trimf(car_density.universe, [0, 0, 5])
car_density['medium'] = fuzz.trimf(car_density.universe, [0, 5, 10])
car_density['high'] = fuzz.trimf(car_density.universe, [5, 10, 10])

traffic_flow['low'] = fuzz.trimf(traffic_flow.universe, [0, 0, 5])
traffic_flow['medium'] = fuzz.trimf(traffic_flow.universe, [0, 5, 10])
traffic_flow['high'] = fuzz.trimf(traffic_flow.universe, [5, 10, 10])

# Çıktı değişkeni için üyelik fonksiyonlarını tanımlama
signal_duration['short'] = fuzz.trimf(signal_duration.universe, [0, 0, 10])
signal_duration['medium'] = fuzz.trimf(signal_duration.universe, [0, 10, 20])
signal_duration['long'] = fuzz.trimf(signal_duration.universe, [10, 20, 30])

# Kuralları tanımlama
rule1 = ctrl.Rule(car_density['low'] & traffic_flow['low'], signal_duration['short'])
rule2 = ctrl.Rule(car_density['low'] & traffic_flow['medium'], signal_duration['medium'])
rule3 = ctrl.Rule(car_density['low'] & traffic_flow['high'], signal_duration['long'])
rule4 = ctrl.Rule(car_density['medium'] & traffic_flow['low'], signal_duration['medium'])
rule5 = ctrl.Rule(car_density['medium'] & traffic_flow['medium'], signal_duration['medium'])
rule6 = ctrl.Rule(car_density['medium'] & traffic_flow['high'], signal_duration['long'])
rule7 = ctrl.Rule(car_density['high'] & traffic_flow['low'], signal_duration['long'])
rule8 = ctrl.Rule(car_density['high'] & traffic_flow['medium'], signal_duration['long'])
rule9 = ctrl.Rule(car_density['high'] & traffic_flow['high'], signal_duration['long'])

# Kontrol sistemi tanımlama
traffic_light_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9])
traffic_light = ctrl.ControlSystemSimulation(traffic_light_ctrl)

# Girdi değerlerini ayarlama
traffic_light.input['car_density'] = 9
traffic_light.input['traffic_flow'] = 9

# Çıktıyı hesaplama
traffic_light.compute()

# Çıktıyı yazdırma
print("Signal Duration:", traffic_light.output['signal_duration'])


Signal Duration: 17.586206896551726
